In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [9]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [13]:
chars = sorted(list(set(text)))
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [15]:
print(len(chars))

65


In [133]:
vocab_size = len(chars)

In [135]:
stoi ={ch:i  for i,ch in enumerate(chars)}
encode = lambda s:[stoi[i] for i in s]

In [137]:
encode("cd")

[41, 42]

In [25]:
itos = {i:ch for i,ch in enumerate(chars)}
decode = lambda l: ''.join(itos[i] for i in l)

In [27]:
decode([41, 42])

'cd'

In [77]:
# Open AI uses tiktoken to encode , here we have 65 possible tokens but tiktoken has 50257 

In [29]:
import torch 

In [31]:
data = torch.tensor(encode(text), dtype=torch.long)

In [35]:
print(data.shape , data.dtype)

torch.Size([1115394]) torch.int64


In [39]:
print(data[:15])

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0])


In [41]:
len(data)

1115394

In [71]:
# split up the data into train and validation sets
n =  int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [139]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [141]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [89]:
torch.randint(120 ,(2,))

tensor([54, 27])

In [143]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print("ix" , ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    # print("x" , x)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # print("y", y)
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [145]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [127]:
a = torch.tensor([[24, 43, 58]])
a = torch.tensor([[24, 43, 58]])

In [147]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        # print(self.token_embedding_table)
    def forward(self, idx, targets):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        B,T,C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits,targets)
        print(logits)
        return logits
m = BigramLanguageModel(vocab_size)
logits , loss = m(xb,yb)
print(loss)
# out = m(a,b)
# print(out.shape) 

tensor([[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
        [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
        [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
        ...,
        [-2.1910, -0.7574,  1.9656,  ..., -0.3580,  0.8585, -0.6161],
        [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
        [-0.6787,  0.8662, -1.6433,  ...,  2.3671, -0.7775, -0.2586]],
       grad_fn=<ViewBackward0>)


ValueError: too many values to unpack (expected 2)

In [149]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [151]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [173]:
batch_size = 32
for steps in range(1000): # increase number of steps for good results...

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5608952045440674


In [177]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


Wiro IOLELERFmesEDOLOngsan anour YCHARTCI t wireat ou t tabr anun hiflybe
We; id ndirouensendsteantr


### Self Attention

In [184]:
torch.randn(4,3,2)

tensor([[[-0.1881,  1.3661],
         [ 0.9206,  0.7364],
         [-0.8994,  1.9893]],

        [[-0.3984, -0.5891],
         [ 1.6731,  0.2370],
         [ 0.4831,  0.8974]],

        [[-0.1061, -0.0132],
         [ 0.5429,  0.5080],
         [-0.1684, -0.8062]],

        [[ 0.6470,  1.1576],
         [ 0.6428, -0.1818],
         [ 0.2294,  0.2868]]])

In [186]:
#bag of words (bow) = averaging

In [192]:
print(nn.Linear(2,3 ,bias=False))

Linear(in_features=2, out_features=3, bias=False)


In [194]:
import torch
import torch.nn as nn

# Define the layer
layer = nn.Linear(2, 3, bias=False)
e)
# Print the layer to see its structure
print(layer)

# Random input of size (batch_size=1, input_features=2)
x = torch.tensor([[1.0, 2.0]])

# Pass through the layer
output = layer(x)

print("Output:", output)


Linear(in_features=2, out_features=3, bias=False)
Output: tensor([[ 1.0993, -0.9380,  1.6415]], grad_fn=<MmBackward0>)


In [198]:
layer2 = nn.Linear(2, 3, bias=False)
# Random input of size (batch_size=1, input_features=2)
x = torch.tensor([[1.0, 2.0]])

# Pass through the layer
output = layer2(x)

print("Output:", output)

Output: tensor([[ 0.2094,  1.2700, -0.8836]], grad_fn=<MmBackward0>)
